In [1]:
# imports
import numpy as np
import pandas as pd
import nltk
import tkinter

In [2]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')

fig, ax = plt.subplots()
x = np.linspace(0,10,1000)
y = np.sin(x)
ax.scatter(x,y, c = x, cmap = 'viridis')

In [3]:
#from google.colab import drive
#drive.mount('/content/drive/')
#%cd "/content/drive/My Drive/NLP2/NLP2_Deep_Generative_Models_for_Text"

In [4]:
from torch.utils.data import Dataset

class AFFRDataset(Dataset):
    def __init__(self, sentences, tokenizer):
        self.sentences = sentences
        self.tokenizer = tokenizer

    def __len__(self):
        """Returns the number of items in the dataset"""
        return len(self.sentences)

    def __getitem__(self, idx):
        """
        Returns the datapoint at index i as a tuple (sentence, label),
        where the sentence is tokenized.
        """
        encoded = self.tokenizer.encode(
            self.sentences[idx], add_special_tokens=True)
        return encoded

In [5]:
from tokenizers import WordTokenizer

# Create the datasets

# Train your tokenizer.
reader = nltk.corpus.BracketParseCorpusReader(".", "02-21.10way.clean")
tree = reader.parsed_sents()[:]
text = [" ".join(line.leaves()).lower() for line in tree]
tokenizer = WordTokenizer(text, max_vocab_size=10000)
EOS = tokenizer.encode('.')

In [6]:
# Check if everything works.
for sentence in text[:5]:
    tokenized = tokenizer.encode(sentence, add_special_tokens=False)
    sentence_decoded = tokenizer.decode(tokenized)
    print('original:', sentence)
    print('tokenized:', tokenized)
    print('decoded:', sentence_decoded)
    print()

original: in an oct. 19 review of `` the misanthrope '' at chicago 's goodman theatre -lrb- `` revitalized classics take the stage in windy city , '' leisure & arts -rrb- , the role of celimene , played by kim cattrall , was mistakenly attributed to christina haag .
tokenized: [4754, 926, 6325, 185, 7745, 6332, 584, 9060, 3, 10, 1161, 2002, 16, 4273, 9063, 22, 584, 3, 3, 8917, 9060, 8556, 4754, 3, 2067, 18, 10, 5363, 8, 1113, 24, 18, 9060, 7826, 6332, 3, 18, 6822, 1744, 5193, 3, 18, 9725, 5944, 1200, 9167, 3, 3, 25]
decoded: in an oct. 19 review of `` the [UNK] '' at chicago 's goodman theatre -lrb- `` [UNK] [UNK] take the stage in [UNK] city , '' leisure & arts -rrb- , the role of [UNK] , played by kim [UNK] , was mistakenly attributed to [UNK] [UNK] .

original: ms. haag plays elianti .
tokenized: [6042, 3, 6826, 3, 25]
decoded: ms. [UNK] plays [UNK] .

original: rolls-royce motor cars inc. said it expects its u.s. sales to remain steady at about 1,200 cars in 1990 .
tokenized: [3, 6

In [7]:
files = ["02-21.10way.clean", "22.auto.clean", "23.auto.clean"]

# Get output dict and remapping of file names to dataset names
data_dict = {}
remap_dict = {"02-21.10way.clean":"train", 
              "22.auto.clean":"validation",
              "23.auto.clean":"test"}

# Parse the data with nltk bracket parser
for file in files:
    reader = nltk.corpus.BracketParseCorpusReader(".", file)
    tree = reader.parsed_sents()[:]
    
    # Assign a dataset dict to train, validation or test
    data_dict[remap_dict[file]] = [" ".join(line.leaves()).lower() for line in tree]

train_data = AFFRDataset(data_dict['train'], tokenizer)
validation_data = AFFRDataset(data_dict['validation'], tokenizer)
test_data = AFFRDataset(data_dict['test'], tokenizer)

In [8]:
import torch.nn as nn

class TextGenerationModel(nn.Module):

    def __init__(self, vocab_size, gru_num_hidden=256, gru_num_layers=2, device='cuda:0',dropout=0):
        super(TextGenerationModel, self).__init__()
        self.embed = nn.Embedding(vocab_size,256)
        self.gru = nn.GRU(256,gru_num_hidden,gru_num_layers,dropout=dropout,batch_first=True)
        self.output = nn.Linear(gru_num_hidden,vocab_size)
        self.device = device
        self.to(device)

    def forward(self, x, hidden=None):
        input = x.to(self.device)
        embed = self.embed(input)
        if(hidden == None):
            out,hidden = self.gru.forward(embed)
        else:
            out,hidden = self.gru.forward(embed,hidden)
        out = self.output(out)

        return out, hidden


In [9]:
from torch.utils.data import DataLoader

def padded_collate(batch):
    """Pad sentences, return sentences and labels as LongTensors."""
    sentences = [sentence[:-1] for sentence in batch]
    labels = [sentence[1:] for sentence in batch]

    lengths = [len(s) for s in sentences]
    max_length = max(lengths)
    # Pad each sentence with zeros to max_length
    padded_sents = [s + [0] * (max_length - len(s)) for s in sentences]
    padded_labels = [s + [0] * (max_length - len(s)) for s in labels]
    return torch.LongTensor(padded_sents), torch.LongTensor(padded_labels)

In [10]:
import os
import time
from datetime import datetime
import argparse

import numpy as np

import torch
import torch.optim as optim

def train(config, train_data, valid_data):

    # Initialize the device which to run the model on
    if(torch.cuda.is_available()):
        device = torch.device("cuda")
        print('Device = CUDA')
    else:
        device = torch.device("cpu")
        print('Device = CPU')
        
    #Paths to save the model and optimizer to
    modelpath = config['model_path']
    optimpath = config['optim_path']

    # Initialize the model that we are going to use
    makeNew = config['new_model']

    #Load in model if necessary
    if(not makeNew and modelpath != ""):
        model = (torch.load(modelpath))
    else:
        model = TextGenerationModel(config['vocab_size'],config['num_hidden'],config['num_layers'],device)

    # Setup the loss and optimizer
    criterion = torch.nn.CrossEntropyLoss(ignore_index=0)
    optimizer = optim.Adam(model.parameters(),config['learning_rate'])

    #Load in the optimizer if necessary
    if(not makeNew and optimpath != ""):
        optimizer.load_state_dict(torch.load(optimpath))

    accs = []
    losses = []
    curr_accs = []
    curr_losses = []
    print_steps = []
    convergence = False
    conv_count = 0
    prev_loss = np.inf

    iteration = 0
    epoch = 0
    while(not convergence):
        print("Epoch: " + str(epoch))
        loss = 0
        accuracy = 0
        for step, (batch_inputs, batch_targets) in enumerate(train_data):
            # Only for time measurement of step through network
            t1 = time.time()
            optimizer.zero_grad()
            targets = batch_targets.to(device)
            out,_ = model.forward(batch_inputs)

            #Calculate loss and accuracy
            curr_loss = 0
            curr_acc = 0
            cor = 0

            seq_length = batch_inputs.shape[1]

            # Get average original lenghts of sentences to divide total loss by
            orig_lengths = torch.mean(torch.argmin(batch_inputs,dim=1).float())

            for i in range(seq_length):
                out_t = out[:,i,:]
                targets_t = targets[:,i]
                curr_loss += criterion.forward(out_t, targets_t)
                preds = (torch.argmax(out_t,dim=1)).long()
                cor += targets_t.eq(preds).sum().item()
            curr_acc = cor/(seq_length * targets.size()[0])
            loss += curr_loss.item()
            accuracy += curr_acc

            curr_loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=config['max_norm'])

            optimizer.step()
            
            curr_accs.append(curr_acc)
            curr_losses.append(curr_loss.item())

            # Just for time measurement
            t2 = time.time()
            examples_per_second = config['batch_size']/float(t2-t1)
            if(iteration % config['print_every'] == 0):
                loss_std = np.std(curr_losses)
                print("[{}] Epoch {:04d}, Train Step {:04d}, Batch Size = {}, Examples/Sec = {:.2f}, "
                    "Avg. Accuracy = {:.2f}, Avg. Loss = {:.3f}, Loss STD = {:.3f}".format(
                        datetime.now().strftime("%Y-%m-%d %H:%M"), epoch, iteration,
                        config['batch_size'], examples_per_second,
                        accuracy/config['print_every'], loss/config['print_every'], loss_std
                ))
                accs.append(accuracy/config['print_every'])
                losses.append(loss/config['print_every'])
                print_steps.append(iteration)
                if(np.abs(prev_loss/config['print_every'] - loss/config['print_every']) < 0.001):
                    conv_count += 1
                else:
                    conv_count = 0
                convergence = conv_count == 5
                prev_loss = loss
                accuracy = 0
                loss = 0
            #Generate text
            if iteration % config['sample_every'] == 0:
                model.eval()
                with torch.no_grad():
                    text = generate_text(model,device,config['sample_strat'],config['sample_temp'])
                    print(str(iteration), ": ", text)
                model.train()
                '''                if(modelpath != ""):
                    torch.save(model,modelpath)
                if(optimpath != ""):
                    torch.save(optimizer.state_dict(),optimpath)'''
            iteration += 1

        #TODO: Validate
        model.eval()
        iter = 0
        for step, (batch_inputs, batch_targets) in enumerate(valid_data):
            targets = batch_targets.to(device)
            out,_ = model.forward(batch_inputs)
            #Calculate loss and accuracy
            curr_loss = 0
            curr_acc = 0
            cor = 0

            seq_length = batch_inputs.shape[1]

            # Get average original lenghts of sentences to divide total loss by
            orig_lengths = torch.mean(torch.argmin(batch_inputs,dim=1).float())

            for i in range(seq_length):
                out_t = out[:,i,:]
                targets_t = targets[:,i]
                curr_loss += criterion.forward(out_t, targets_t)
                preds = (torch.argmax(out_t,dim=1)).long()
                cor += targets_t.eq(preds).sum().item()
            curr_acc = cor/(seq_length * targets.size()[0])
            loss += curr_loss.item()
            accuracy += curr_acc
            curr_accs.append(curr_acc)
            curr_losses.append(curr_loss.item())
            iter += 1
        loss_std = np.std(curr_losses)
        print("Epoch {:04d}, Validation, Avg. Accuracy = {:.2f}, Avg. Loss = {:.3f}, Loss STD = {:.3f}".format(
                epoch, accuracy/iter, loss/iter, loss_std))
        model.train()
        epoch += 1
        if(epoch == config['epochs'] or convergence):
            break

    print('Done training.')
    print(accs)
    print(losses)
    print(print_steps)

In [11]:
import torch.nn.functional as F

def generate_text(model,device,sampling_strat='max',temperature=1, starting_text=[1],ksize=1):
    assert sampling_strat in ('max', 'rand')
    
    # Start with encoded text
    start = np.array(starting_text)
    text = list(start) #This stores the eventual output
    current = torch.from_numpy(start).long().unsqueeze(dim=0) 

    #The initial step
    input = current.to(device)
    output,hidden = model.forward(input)
    current = output[0,-1,:].squeeze()
    if(sampling_strat == 'max'):
        guess = torch.argmax(current).unsqueeze(0)
    elif(sampling_strat == 'rand'):
        guess = torch.multinomial(F.softmax(temperature*current,dim=0),1)
    text.append(guess.item())
    input = guess.unsqueeze(0)

    #Now that we have an h and c, we can start the loop
    i = 0
    while(i < 100):
        output,hidden = model.forward(input,hidden)
        current = output.squeeze()
        if(sampling_strat == 'max'):
            guess = torch.argmax(current).unsqueeze(0)
        elif(sampling_strat == 'rand'):
            guess = torch.multinomial(F.softmax(temperature*current,dim=0),1)
        text.append(guess.item())
        input = guess.unsqueeze(0)
        i += 1
        if(guess.item() == 2):
            break

    return tokenizer.decode(text)

In [12]:
'''config = {}

# Model params
config['model_path'] = "models\\trump_model.txt"
config['optim_path']="models\\trump_optim.txt"
config['new_model']=True
config['num_hidden']=128
config['num_layers']=2
config['vocab_size']=tokenizer.vocab_size

# Training params
config['batch_size']=64
config['learning_rate']=2e-3

# It is not necessary to implement the following three params, but it may help training.
config['learning_rate_decay']=0.96
config['learning_rate_step']=5000
config['dropout_keep_prob']=1
config['epochs']=10
config['max_norm']=5.0

# Misc params
config['print_every']=100
config['sample_every']=100
config['sample_strat']='rand'
config['sample_temp']=1.5

# Create dataloaders
train_loader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True, collate_fn=padded_collate)
valid_loader = DataLoader(validation_data, batch_size=256, shuffle=False, collate_fn=padded_collate)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False, collate_fn=padded_collate)

# Train the model
train(config, train_loader, valid_loader)'''

'config = {}\n\n# Model params\nconfig[\'model_path\'] = "models\\trump_model.txt"\nconfig[\'optim_path\']="models\\trump_optim.txt"\nconfig[\'new_model\']=True\nconfig[\'num_hidden\']=128\nconfig[\'num_layers\']=2\nconfig[\'vocab_size\']=tokenizer.vocab_size\n\n# Training params\nconfig[\'batch_size\']=64\nconfig[\'learning_rate\']=2e-3\n\n# It is not necessary to implement the following three params, but it may help training.\nconfig[\'learning_rate_decay\']=0.96\nconfig[\'learning_rate_step\']=5000\nconfig[\'dropout_keep_prob\']=1\nconfig[\'epochs\']=10\nconfig[\'max_norm\']=5.0\n\n# Misc params\nconfig[\'print_every\']=100\nconfig[\'sample_every\']=100\nconfig[\'sample_strat\']=\'rand\'\nconfig[\'sample_temp\']=1.5\n\n# Create dataloaders\ntrain_loader = DataLoader(train_data, batch_size=config[\'batch_size\'], shuffle=True, collate_fn=padded_collate)\nvalid_loader = DataLoader(validation_data, batch_size=256, shuffle=False, collate_fn=padded_collate)\ntest_loader = DataLoader(test

# Sentence VAE

In [13]:
import torch
from SentenceVAE import run_epoch
from SentenceVAE import SentenceVAE
from torch.optim import Adam

if(torch.cuda.is_available()):
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

epochs = 10
zdim = 13
batch_size = 4
vocab_size = tokenizer.vocab_size

# Create dataloaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=padded_collate)
valid_loader = DataLoader(validation_data, batch_size=256, shuffle=False, collate_fn=padded_collate)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False, collate_fn=padded_collate)

model = SentenceVAE(vocab_size, z_dim=zdim) 
model.to(device)
#sampled = model.sample(9,device)[0]

optimizer = Adam(model.parameters())

train_curve, val_curve = [], []

for epoch in range(epochs):
    elbos = run_epoch(model, (train_loader, valid_loader), optimizer, device)
    train_elbo, val_elbo = elbos
    train_curve.append(train_elbo)
    val_curve.append(val_elbo)
    print(f"[Epoch {epoch}] train elbo: {train_elbo} val_elbo: {val_elbo}")
    
    #sampled = model.sample(9, device)[0]

save_elbo_plot(train_curve, val_curve, 'elbo.pdf')

RuntimeError: CUDA out of memory. Tried to allocate 1.14 GiB (GPU 0; 3.95 GiB total capacity; 2.42 GiB already allocated; 1.02 GiB free; 2.44 GiB reserved in total by PyTorch)